# KdV with box initial data

Consider solving
\begin{align*}
u_t + u_{xxx} &= u u_x,\\
u(x,0) &= \begin{cases} 0 & 0 < x < \pi,\\
1/2 & \pi < x < 2 \pi. \end{cases}
\end{align*}
It follows that
$u(x,t) = -q(6^{-1/2} x, 6^{-3/2} t)$ where $q$ solves
\begin{align*}
    q_t + 6 q q_x + q_{xxx} = 0,
\end{align*}
with an appropriately modified inital condition.

In [1]:
using PeriodicKdV, Plots

In [4]:
prec = 500
sp(x) = BigFloat(x,precision = prec)
spc(x) = BigFloat(real(x),precision = prec)+1im*BigFloat(imag(x),precision = prec)

ω = pi/sqrt(6.0 |> sp)
T = 2.0*pi/sqrt(6 |> sp)
h = .5;

Δ1 = λ -> 2*cos((T-ω)*sqrt(λ-h))*cos(ω*sqrt(λ))
Δ2 = λ -> -(2*λ-h)/(sqrt(λ)*sqrt(λ-h))*sin(ω*sqrt(λ))*sin((T-ω)*sqrt(λ-h))
Δ = λ ->  Δ1(λ |> complex) + Δ2(λ |> complex) |> real
T22 = λ -> cos(ω*sqrt(λ))*cos((T-ω)*sqrt(λ-h)) - sqrt(λ-h)/sqrt(λ)*sin((T-ω)*sqrt(λ-h))*sin(ω*sqrt(λ))
T11 = λ -> Δ(λ) - T22(λ)
y2sqrt = λ -> 1/sqrt(λ)*cos((T-ω)*sqrt(λ-h))*sin(ω*sqrt(λ)) + 1/sqrt(λ - h)*cos(ω*sqrt(λ))*sin((T-ω)*sqrt(λ-h))

y2 = λ -> y2sqrt(λ^2)
dy2 = λ -> imag(y2(λ + 1im*spc(1e-50))/sp(1e-50))

#31 (generic function with 1 method)

In [5]:
function Newton(f,df,x0,eps,nmax)
    x = x0
    for i = 1:nmax
        δ = -f(x)/df(x)
        if abs(δ) < eps
            return x + δ
        else
            x = x + δ
        end
    end
    if abs(f(x)) > 1e-15
        @warn "failed to converge"
    end
    return x
end

function Bisection(f,A,B,nmax)
    a = A; b = B;
    if abs(f(a)) < 1e-50
        return a
    elseif abs(f(b)) < 1e-50
        return b
    end
    c = a;
    for i = 1:nmax
        fa = f(a); 
        fb = f(b);
        c = (a + b)/2;
        fc = f(c)
        #println((fa,fc,fb))
        #println((a,c,b))
        if fa*fc > 0
            a = c
        else
            b = c
        end
    end
    c
end

Bisection (generic function with 1 method)

In [6]:
tm = λ -> sqrt(Δ(λ)^2 - 4.0 |> complex) - (T11(λ) - T22(λ))
tp = λ -> sqrt(Δ(λ)^2 - 4.0 |> complex) + (T11(λ) - T22(λ))

target_g = 1000
z1 = Newton(y2,dy2,sqrt(1.5 |> sp ),1e-32,100)
zs = [z1];
for i = 1:target_g
    x0 = zs[end] + pi/T
    x0 = Newton(y2,dy2,x0,1e-50,100)
    zs = vcat(zs,[x0])
end
oldzs = copy(zs)
zs = zs.^2

gs = [Bisection(λ -> Δ(λ) - 2,0.00001 |> sp ,zs[1] |> sp ,60) |> Float64, Bisection(λ -> Δ(λ) + 2,0.00001 |> sp,zs[1] |> sp,60) |> Float64] 
for i = 0:target_g-1
    gs = vcat(gs,[Bisection(λ -> Δ(λ) + 2, zs[i+1] |> sp, zs[i+2] |> sp,60) |> Float64, Bisection(λ -> Δ(λ) - 2,zs[i+1] |> sp,zs[i+2] |> sp,80) |> Float64 ])
end
gs = gs |> sort

α1 = gs[1]
zs = map(Float64,hcat(copy(zs) .- α1,sign.(map(tp,zs) - map(tm,zs) |> real)));
gaps = hcat(gs[2:2:end-1],gs[3:2:end]) .- α1;

In [11]:
g = 30
S = HyperellipticSurface(gaps[1:g,:],zs[1:g,:],α1,100);
BA = BakerAkhiezerFunction(S,200.;tols=[1e-4,false],iter = 20,max_pts = 10, show_flag = false, choose_points = [10,10,10,10,3]);
u = (x,t) -> -KdV(BA,x/sqrt(6),t*6^(-3/2),1e-4)

#49 (generic function with 1 method)

In [12]:
u(1.0,0.0)

0.0018944572867880538 + 2.9925547723134244e-17im